In [1]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/analysis/"
img_DIR_P = "../data/images/players/"
img_DIR_T = "../data/images/teams/"
data_DIR = "../data/misc/"
box_DIR = "../data/box/"
aio_DIR = "../data/all_in_one_metrics/"

In [2]:
dft = pd.read_csv("../data/NBA_teams_colors_logos.csv")
teams_dict = dict(zip(dft['teamTricode'], dft['nameTeam']))

In [14]:
# file = "sample_playoff_series_consensus_results.csv"
file1 = "all_teams_series_probabilities_V1.csv"
file2 = "all_teams_series_probabilities_V2.csv"
df1_1 = pd.read_csv(data_DIR + file1)
df1_2 = pd.read_csv(data_DIR + file2)
df1_2.iloc[:,4:] = df1_2.iloc[:,4:]*100
df1 = pd.concat([df1_1,df1_2])
df1 = df1.reset_index(drop=True)
df1.loc[df1["Model"].str.contains("predict_series_ridge"),"Model"]="pred_ridge"

In [4]:
df1.columns

Index(['Team', 'Opponent', 'Series_home', 'Model', 'OverallWin', 'OverallLoss',
       'WinIn4', 'WinIn5', 'WinIn6', 'WinIn7', 'LoseIn7', 'LoseIn6', 'LoseIn5',
       'LoseIn4'],
      dtype='object')

In [5]:
df1.Model.unique()

array(['home_court', 'lebron', 'epm', 'lazy_lebron', 'pred_epm', 'dpm',
       'box', 'bpm', 'bpm_larapm', 'raptor', 'xrapm', 'ebron', 'bpm_rapm',
       'multi_lebron', 'rs_ratings', 'combined', 'ridge', 'lasso',
       'elasticnet', 'xgboost', 'naive_bayes', 'neural_network',
       'consensus', 'predict_series_ridge'], dtype=object)

In [16]:
models = ['home_court', 'lebron', 'epm', 'lazy_lebron', 'pred_epm', 'dpm',
       'box', 'bpm', 'bpm_larapm', 'raptor', 'xrapm', 'ebron', 'bpm_rapm',
       'multi_lebron', 'rs_ratings', 'combined', 'ridge', 'lasso',
       'elasticnet', 'xgboost', 'naive_bayes', 'neural_network',
       'consensus','predict_series_ridge']
models = ['pred_ridge','lebron', 'epm', 'dpm','multi_lebron', 'pred_epm', 'bpm', 'raptor', 'xrapm',]

In [17]:
df1e = df1[df1["Model"].isin(models)].reset_index(drop=True)
df1e['Model'] = pd.Categorical(df1e['Model'],categories=models,ordered=True)
df1e.iloc[:,4:] = df1e.iloc[:,4:].round(2)
df2e = df1e.drop(columns="Series_home")

In [19]:
df2e.to_parquet("../data/shiny/playoff_model_probabilities_2024.parquet")
exp_DIR = "../../shinyNBA/data/"
df2e.to_parquet(exp_DIR + "playoff_model_probabilities_2024.parquet")

In [ ]:
df2 = df1e[df1e["Series_home"] == 1]

In [ ]:
df2["Team"].unique()

In [ ]:
team_h = ["BOS","NYK","IND","HOU","LAL","DEN","OKC","OKC","CLE","CLE"]
team_a = ["ORL","DET","MIL","GSW","MIN","LAC","MEM","DAL","ATL","MIA"]

In [ ]:
dfa = []
for h,a in zip(team_h,team_a):
    df3 = df2[(df2["Team"] == h) & (df2["Opponent"] == a)]
    df3 = df3.sort_values("Model")
    dfa.append(df3)
df4 = pd.concat(dfa)
df4.iloc[:,4:] = df4.iloc[:,4:].round(2)
df5 = df4.drop(columns = ["Series_home"])
df5 = df5.reset_index(drop=True)

In [ ]:
r = range(len(team_h))
# r = range(1)
for i in r:
    ds = df5[(df5["Team"] == team_h[i]) & (df5["Opponent"] == team_a[i])]
    ds = ds.reset_index(drop=True)
    ds["Team"] = ds["Team"].map(teams_dict)
    ds["Opponent"] = ds["Opponent"].map(teams_dict)
    title = f"**Playoff Series Predictions using various Metrics**"
    subtitle = f'**{team_h[i]} vs {team_a[i]}**'
    t = (
        GT(ds)
        .tab_header(
            title=md(title),
            subtitle=md(subtitle)
        )
        .tab_source_note(source_note="@knarsu3,@sradjoker | source:bball-index,darko,dunksandthrees,bbref,xrapm,neilpaine")
        .cols_label(
            Team = "Home",
            Opponent = "Away",
            OverallWin = "Overall Win",
            OverallLoss = "Overall Loss",
            WinIn4="4", 
            WinIn5="5", 
            WinIn6="6", 
            WinIn7="7", 
            LoseIn7="7",
            LoseIn6="6", 
            LoseIn5="5", 
            LoseIn4="4",
        )
        .fmt_image(
            columns=["Team","Opponent"],
            path = img_DIR_T,
            file_pattern="{}.png",
            height = 40
        )
        .tab_spanner(label="W", columns=['WinIn4', 'WinIn5', 'WinIn6', 'WinIn7'])
        .tab_spanner(label="L", columns=['LoseIn7','LoseIn6', 'LoseIn5', 'LoseIn4'])
        .cols_align(align="center")
        .cols_align(align="left", columns=["Team","Oppoenent"])
        .data_color(
            domain = [0.0,100.0],
            columns=['OverallWin','OverallLoss'],
            palette="RdYlGn",
        )
        .data_color(
            domain = [0.0,50.0],
            columns=[ 'WinIn4', 'WinIn5', 'WinIn6', 'WinIn7','LoseIn7','LoseIn6', 'LoseIn5', 'LoseIn4'],
            palette="RdYlGn",
        )
        .tab_options(
            heading_title_font_size="150%",
            heading_subtitle_font_size="150%",
            # table_background_color="ghostwhite",
            table_background_color="snow",
            column_labels_font_size="105%",
            column_labels_font_weight='bold',
            row_group_font_weight='bold',
            row_group_background_color="#E5E1D8",
            table_font_size=10,
            table_font_names="Consolas", 
            data_row_padding = "3px",
            # table_margin_left = 7,
            # table_margin_right = 0,
        )
    )
    t.save(fig_DIR + f"playoff_model_table_2024_{(i+1):02d}.png",scale=3,web_driver="firefox")
t